# 0. Format

## 0.1 data template

In [ ]:
dp_template = {
    'meta_data': {
        'dataset': None,
        'id': None,
        'eval_func': None
    },
    'input': {
        'text': None,
        'file': None,
        'others': {
            'constraints': [],
            'instructions': {'system': None, 'user': None},
            'references': []
        }
    },
    'output': None
}

## 0.2 Workflow

In [ ]:
# class Model():
#     def __init__(self, model_config):
#         pass

#     def make_prompts(self, batch_input):
#         pass

#     def batch_predict(self, batch_prompt):
#         pass

#     def post_processing(self, batch_output):
#         pass

## 0.3 Data Clean

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# neocoder
import json
from copy import deepcopy

with open("/content/drive/MyDrive/Creativity Benchmark/raw/neocoder/datasets/CodeForce/NeoCoder/NeoCoder.json", "r") as f:
  data = json.load(f)

with open("/content/drive/MyDrive/Creativity Benchmark/raw/neocoder/datasets/CodeForce/NeoCoder/human_solution_techniques.json", "r") as f:
  human_solution = json.load(f)

with open("/content/drive/MyDrive/Creativity Benchmark/raw/neocoder/datasets/CodeForce/NeoCoder/test_cases_annotated.json", "r") as f:
  test_case_list = json.load(f)

# convert test_case_list to dict
test_case = {}
for item in test_case_list:
  test_case[item['problem_id']] = {'input': item['input'], 'output': item['output']}

processed_result = []
for item in data:
  dp_template_cp = deepcopy(dp_template)
  dp_template_cp['meta_data']['dataset'] = 'NeoCoder'
  dp_template_cp['meta_data']['id'] = item['problem_id']

  dp_template_cp['input']['text'] = item['problem_statements'][0]
  dp_template_cp['input']['others']['constraints'] = item['constraints_list']
  dp_template_cp['input']['others']['instructions']['system'] = """You are a Python code generator, only return the import and python function.
Input will be an very detailed description of task, output will be the code.
The input will be from command line, and the output will be printed to the
console as well. Your result will be solely a function named solve(), and
do not call this function in your code. Make sure the code is free of bug
and can pass the test cases provided. You can use any library you want. The
test cases are provided in the code. Do not call the solve() function in
your code."""
  # note that we need to format variable constraints and input question
  dp_template_cp['input']['others']['instructions']['user'] = """Programming constraints: Do not use the following techniques: {constraints}.\n{question}"""

  dp_template_cp['input']['others']['references'] = {'human_solutions': human_solution[item['problem_id']], "test_cases": test_case[item['problem_id']]}
  processed_result.append(dp_template_cp)

with open("/content/drive/MyDrive/Creativity Benchmark/processed/neocoder.json", "w") as f:
  json.dump(processed_result, f, indent=4)

In [ ]:
# CS4
import re
import json
import pandas as pd
from copy import deepcopy

story_constraints = pd.read_csv("/content/drive/MyDrive/Creativity Benchmark/raw/cs4/Story-based Constraints.csv")
instruction_constraints = pd.read_csv("/content/drive/MyDrive/Creativity Benchmark/raw/cs4/Instruction-based Constraints.csv")
story_constraints['constraint_list'] = story_constraints['Constraints'].apply(lambda x: re.split(r"\d+. ", x))
story_constraints['constraint_list'] = story_constraints['constraint_list'].apply(lambda x: [i.strip() for i in x if i != ''])
instruction_constraints['constraint_list'] = instruction_constraints['Constraints'].apply(lambda x: re.split(r"\d+. ", x))
instruction_constraints['constraint_list'] = instruction_constraints['constraint_list'].apply(lambda x: [i.strip() for i in x if i != ''])

story_constraints = story_constraints[story_constraints['Number of Constraints'] == 39]
instruction_constraints = instruction_constraints[instruction_constraints['Number of Constraints'] == 39]

processed_result = []
datasets = {'story': story_constraints, 'instruction': instruction_constraints}
for dataset_name, dataset in datasets.items():
  for idx, item in dataset.iterrows():
    dp_template_cp = deepcopy(dp_template)
    dp_template_cp['meta_data']['dataset'] = "CS4"
    dp_template_cp['meta_data']['id'] = dataset_name + "_" + str(idx)
    dp_template_cp['input']['text'] = item['Instruction ']
    dp_template_cp['input']['others']['constraints'] = item['constraint_list']
    dp_template_cp['input']['others']['instructions']['system'] = """You are a writing expert. Come up with a creative story in no more than 500 words based on the below instruction."""
    dp_template_cp['input']['others']['instructions']['user'] = """Instruction: {question}\nConstraints: {constraints}"""
    processed_result.append(dp_template_cp)

with open("/content/drive/MyDrive/Creativity Benchmark/processed/cs4.json", "w") as f:
  json.dump(processed_result, f, indent=4)


In [ ]:
%pip install datasets


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/215M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/109M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# suri
import datasets
import json
from copy import deepcopy

data = datasets.load_dataset("chtmp223/suri")

processed_result = []
for split in ["train", "dev", "test"]:
  for item in data[split]:
    dp_template_cp = deepcopy(dp_template)
    dp_template_cp['meta_data']['dataset'] = "suri"
    dp_template_cp['meta_data']['id'] = item['id']
    dp_template_cp['input']['text'] = item['main_ins']
    dp_template_cp['input']['others']['constraints'] = [x.strip() for x in item['cons_chosen'].split('- ') if x != ""]
    dp_template_cp['input']['others']['instructions']['user'] = """Main Goal: {question}\nConstraints: {constraints}"""

    processed_result.append(dp_template_cp)

with open("/content/drive/MyDrive/Creativity Benchmark/processed/suri.json", "w") as f:
  json.dump(processed_result, f, indent=4)

In [ ]:
# creative_math
import json
from copy import deepcopy

with open("/content/drive/MyDrive/Creativity Benchmark/raw/creative_math/data/subset.json", "r") as f:
  data = json.load(f)

processed_result = []

for item in data:
  dp_template_cp = deepcopy(dp_template)
  dp_template_cp['meta_data']['dataset'] = "creative_math"
  dp_template_cp['meta_data']['id'] = item['competition_id'] + "_" + str(item["problem_id"])

  dp_template_cp['input']['text'] = item['problem']
  dp_template_cp['input']['others']['instructions']['system'] = """Criteria for evaluating the difference between two mathematical solutions include:
1. If the methods used to arrive at the solutions are fundamentally different, such as algebraic manipulation versus geometric reasoning, they can be considered distinct;
2. Even if the final results are the same, if the intermediate steps or processes involved in reaching those solutions vary significantly, the solutions can be considered different;
3. If two solutions rely on different assumptions or conditions, they are likely to be distinct;
4. A solution might generalize to a broader class of problems, while another solution might be specific to certain conditions. In such cases, they are considered distinct;
5. If one solution is significantly simpler or more complex than the other, they can be regarded as essentially different, even if they lead to the same result."""
  dp_template_cp['input']['others']['instructions']['user'] = """Given the following mathematical problem:
{question}
And some typical solutions:
{solutions}
Please output a novel solution distinct from the given ones for this math problem."""

  dp_template_cp['input']['others']['references'] = {"solutions": [solution for solution in item['solutions'].values()]}

  processed_result.append(dp_template_cp)

with open("/content/drive/MyDrive/Creativity Benchmark/processed/creative_math.json", "w") as f:
  json.dump(processed_result, f, indent=4)

In [ ]:
# macgyver
import pandas as pd
import json
from copy import deepcopy

data = pd.read_excel("/content/drive/MyDrive/Creativity Benchmark/raw/macgyver/data/MacGyver/problem_solution_pair.xlsx")
human_data = pd.read_excel("/content/drive/MyDrive/Creativity Benchmark/raw/macgyver/data/MacGyver/additional_human_solutions.xlsx")

processed_result = []

for idx, item in data.iterrows():
  dp_template_cp = deepcopy(dp_template)
  dp_template_cp['meta_data']['dataset'] = "macgyver"
  dp_template_cp['meta_data']['id'] = item['ID']
  dp_template_cp['input']['text'] = item['Problem'] # constraints included
  dp_template_cp['input']['others']['instructions']['user'] = """{question}
Give a feasible solution very concisely . Note that some tools are not useful , so please analyze the affordance of each presented object , and rule out unnecessary ones first .
Use the following format:
1. List the affordance of presented items and whether they are useful
2. Summary: list useful tools
3. If the problem is solvable under all these constraints , write the solution . Use step1 , step2 , etc , and mention the tools to achieve each step . Use as few steps as possible and the answer should ideally be less than 100 words.

If you cannot find a feasible solution , just say that it is not possible and give a very short justification ."""

  dp_template_cp['input']['others']['references'] = {'solution': item['Solution'], 'solvable': item['Solvable?'], "unconventional": item['Unconventional?'], 'human_annotated_label': item['Label']}

  processed_result.append(dp_template_cp)

with open("/content/drive/MyDrive/Creativity Benchmark/processed/macgyver.json", "w") as f:
  json.dump(processed_result, f, indent=4)
